# 1. Hugging Face and Transformers

Transformers pros:

    Automatic model downloads
    Code snippets available
    Ideal for experimentation and learning

Transformers cons:

    Requires solid understanding of ML and NLP
    Coding and configuration skills are necessary



In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium", padding_side='left')
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# source: https://huggingface.co/microsoft/DialoGPT-medium

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last output tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

>> User: Hello


DialoGPT: Hello! :D


>> User: How are you


DialoGPT: I'm good, how are you?


...

DialoGPT: That's good


>> User: OK


DialoGPT: I'm good


>> User: Do you know my name


DialoGPT: I do


# 2. LangChain

In [3]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="microsoft/DialoGPT-medium", task="text-generation", pipeline_kwargs={"max_new_tokens": 200, "pad_token_id": 50256},
)

from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

Device has 2 GPUs available. Provide device={deviceId} to `from_model_id` to use availableGPUs for execution. deviceId is -1 (default) for CPU and can be a positive integer associated with CUDA device id.


Question: What is electroencephalography?

Answer: Let's think step by step.I'm not a neuroscientist, but I'm pretty sure it's a branch of neuroscience.


# 3. Llama.cpp

[Llama.cpp](https://github.com/ggerganov/llama.cpp) is a C and C++ based inference engine for LLMs, optimized for Apple silicon and running Meta’s Llama2 models.

Once we clone the repository and build the project, we can run a model with: `$ ./main -m /path/to/model-file.gguf -p "Hi there!"`

Llama.cpp Pros:

    Higher performance than Python-based solutions
    Supports large models like Llama 7B on modest hardware
    Provides bindings to build AI applications with other languages while running the inference via Llama.cpp.

Llama.cpp Cons:

    Limited model support
    Requires tool building

# 4. Llamafile

[Llamafile](https://github.com/Mozilla-Ocho/llamafile), developed by Mozilla, offers a user-friendly alternative for running LLMs. Llamafile is known for its portability and the ability to create single-file executables.

Once we download llamafile and any GGUF-formatted model, we can start a local browser session with: `$ ./llamafile -m /path/to/model.gguf`

Llamafile pros:

    Same speed benefits as Llama.cpp
    You can build a single executable file with the model embedded

Llamafile cons:

    The project is still in the early stages
    Not all models are supported, only the ones Llama.cpp supports.

# 5. Ollama

[Ollama](https://ollama.com/) is a more user-friendly alternative to Llama.cpp and Llamafile. You download an executable that installs a service on your machine. Once installed, you open a terminal and run: `$ ollama run llama2`

Ollama pros:

    Easy to install and use.
    Can run llama and vicuña models.
    It is really fast.

Ollama cons:

    Provides limited model library.
    Manages models by itself, you cannot reuse your own models.
    Not tunable options to run the LLM.
    No Windows version (yet).

[model library](https://ollama.com/library)

# 6. GPT4ALL

GPT4ALL is an easy-to-use desktop application with an intuitive GUI. It supports local model running and offers connectivity to OpenAI with an API key. It stands out for its ability to process local documents for context, ensuring privacy.

GPT4ALL Pros:

    Polished alternative with a friendly UI
    Supports a range of curated models

GPT4ALL Cons:

    Limited model selection
    Some models have commercial usage restrictions

# references

[6 Ways For Running A Local LLM (how to use HuggingFace)](https://semaphoreci.com/blog/local-llm)